In [1]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px

/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [5]:
m = pd.read_excel("/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_analysis.xlsx", sheet_name='Master')

In [6]:
def hypno_status(subject):
    

,Hypnograms,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Subject,Recording,Hypno,Scored?,Complete?,Scored By
1,ACR_14,swi-bl,swi-bl_chunk1,yes,yes,KD
2,ACR_13,laser1,laser1_chunk1,yes,yes,KD
3,NaN,NaN,NaN,NaN,NaN,NaN
4,Spike Sorting,NaN,NaN,NaN,NaN,NaN
5,Subject,Experiement,Recordings,Sorted,Curated By,NaN
6,ACR_13,laser1,"laser1-bl, laser1, laser1-post",N,NaN,NaN


In [2]:
import acr.info_pipeline as aip
inf = aip.load_subject_info('ACR_14')

In [25]:
subject = 'ACR_14'
t = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_14/hypnograms/hypno_sdpi-bl_chunk1.txt'
exp = t.split('/')[-1].split('.')[0].split('_')[1:][0]
chunk = t.split('/')[-1].split('.')[0].split('_')[1:][1]

config_path = Path(f'/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/{subject}/config-files/{subject}_sleepscore-config_{exp}-{chunk}.yml')
config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
config['tStart']

0

In [ ]:
ACR_14_sleepscore-config_sdpi-bl-chunk1
ACR_14_sleepscore-config_sdpi-bl-chunk1

In [44]:
subject = 'ACR_14'
root = Path(f'/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/{subject}/hypnograms/')
hyp_info = {}
for rec in inf['recordings']:
    name_root = f'hypno_{rec}_chunk'
    hyp_info[rec] = {}
    hyp_info[rec]['starts'] = []
    hyp_info[rec]['ends'] = []
    for file in root.glob(f'{name_root}*'):
        file = str(file)
        exp = file.split('/')[-1].split('.')[0].split('_')[1:][0]
        chunk = file.split('/')[-1].split('.')[0].split('_')[1:][1]
        config_path = Path(f'/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/{subject}/config-files/{subject}_sleepscore-config_{exp}-{chunk}.yml')
        config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
        hyp_info[rec]['starts'].append(config['tStart'])
        hyp_info[rec]['ends'].append(config['tEnd'])

In [47]:
hyp_info['swi-bl']

{'starts': [7200, 0, 14400, 0, 0], 'ends': [14400, 7200, 21600, 7200, 7200]}

In [46]:
for rec in hyp_info.keys():
    if not hyp_info[rec]['starts']:
        print(f'No hypnograms for {rec}')
        pass
    else:
        starts = sorted(hyp_info[rec]['starts'])
        ends = sorted(hyp_info[rec]['ends'])
        print(rec)
        print(starts)
        print(ends)
        start = starts[0]
        end = ends[-1]
        print(f'{rec} covers: {start} - {end}')
        duration = end - start
        print(f'{rec} duration: {duration}')

No hypnograms for plugged_test
No hypnograms for mon1
sdpi-bl
[0, 7200, 14400]
[7200, 14400, 21600]
sdpi-bl covers: 0 - 21600
sdpi-bl duration: 21600
No hypnograms for sdpi-post
No hypnograms for short1-iso
No hypnograms for mon2
No hypnograms for short1-plug
No hypnograms for full-day
swi-bl
[0, 0, 0, 7200, 14400]
[7200, 7200, 7200, 14400, 21600]
swi-bl covers: 0 - 21600
swi-bl duration: 21600
sdpi
[0, 7200, 14400, 21600, 28800, 36000]
[7200, 14400, 21600, 28800, 36000, 43200]
sdpi covers: 0 - 43200
sdpi duration: 43200
No hypnograms for ep-test1
swi
[0, 7200, 14400]
[7200, 14400, 21600]
swi covers: 0 - 21600
swi duration: 21600
No hypnograms for laser1-post1
No hypnograms for short3-plug
No hypnograms for isolated_test
No hypnograms for short2-plug
laser1
[13999, 17599, 21199, 24799, 28399, 31999]
[17599, 21199, 24799, 28399, 31999, 35599]
laser1 covers: 13999 - 35599
laser1 duration: 21600
No hypnograms for short2-iso
swi-sd
[0, 7200, 14400]
[7200, 14400, 21600]
swi-sd covers: 0 - 2

In [1]:
import acr.info_pipeline as aip
inf = aip.load_subject_info('ACR_14')

In [3]:
inf['rec_times']['swi-bl']

{'duration': 86464.616425,
 'end': '2022-10-07T09:02:23.616424',
 'start': '2022-10-06T09:01:18.999999'}

In [4]:
inf['rec_times']['swi-sd']

{'duration': 17884.876654,
 'end': '2022-10-07T14:06:20.876653',
 'start': '2022-10-07T09:08:15.999999'}

In [5]:
86464+17884

104348